In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, ImageMessage, TextSendMessage
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")
app_1 = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )
# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "素材/問題一/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
    
    # 呼叫個人用戶名，與用戶第一次接觸打招呼
    user_profile = line_bot_api.get_profile(event.source.user_id)
    hello = {
      "type": "text",
      "text": "哈囉!" + str(user_profile.display_name) + "\n歡迎您使用「履視步爽」"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
    
        
    #綁定 rich_menu
    linkRichMenuId = open("素材/rich_menu_4/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)


    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )
    
    #擷取tag訊息
    dn = "ID_message/user_event.txt"
    user_event = {}
    with open(dn, "a") as myfile:
        user_event["user_id"] = vars(user_profile)["user_id"]
        user_event["tag"] = "user of first entering"
        user_event["time"] = event.timestamp
        user_event["product_ID"] = "now to choose"
        myfile.write(json.dumps(user_event,sort_keys=True))
        myfile.write('\r\n')

In [5]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

譯json後，將消息回傳給用戶

無則不發送訊息

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage, PostbackEvent
)
from os import listdir
# from urllib.parse import parse_qs

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    
    # 指定要列出素材的目錄
    mypath = "素材/"
    # 取得所有檔案與子目錄名稱
    list_text = listdir(mypath)
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
    user_profile = line_bot_api.get_profile(event.source.user_id)


    if event.message.text in list_text:
        
        replyJsonPath = "素材/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = event.message.text
            user_event["time"] = event.timestamp
            user_event["product_ID"] = "now to choose"
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
    
    else:
        pass
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = "no send out message"
            user_event["time"] = event.timestamp
            user_event["product_ID"] = "now to choose"
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')



In [6]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送
    擷取tag訊息

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送
    擷取tag訊息

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 
import time

@handler.add(PostbackEvent)
def process_postback_event(event):
    

    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        #讀取其reply.json，轉譯成消息
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
         
        top_line = {
          "type": "text",
          "text": "已記下，放在心上，開始推送"
        }
        result_message_array.insert(0, TextSendMessage.new_from_json_dict(top_line))
  
        ##發送訊息
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
            
        )
        
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = event.timestamp
            user_event["product_ID"] = "now to choose"
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')
            
    elif 'menu' in query_string_dict:
    
        #讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        #讀取其reply.json，轉譯成消息
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
         
        top_line = {
          "type": "text",
          "text": "已記下，放在心上，開始推送"
        }
        result_message_array.insert(0, TextSendMessage.new_from_json_dict(top_line))
  
        #發送訊息
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        ) 
        
        #擷取tag訊息
        dn = "ID_message/user_event.txt"
        user_event = {}
        with open(dn, "a") as myfile:
            user_event["user_id"] = vars(user_profile)["user_id"]
            user_event["tag"] = query_string_dict.get('tag')[0]
            user_event["time"] = event.timestamp
            user_event["product_ID"] = "now to choose"
            myfile.write(json.dumps(user_event,sort_keys=True))
            myfile.write('\r\n')

In [7]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。
擷取照片的主顏色

'''
#!pip install clarifai
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
import os

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
        
    message_content = line_bot_api.get_message_content(event.message.id)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    #擷取照片的主顏色
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
    #路徑為相對路徑
    Image_shoes = './images/'+event.message.id+'.jpg'
    # ======================= 官方固定格式 ===============================
    app = ClarifaiApp(api_key='0f82436fcad44c1cb8953ee52d563ef1')
    # model = app.public_models.general_model
    model = app.models.get('color')
    # picnames = pic_list('D:\clarifai\pictures')
    image = ClImage(filename=Image_shoes)
    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    # ===================================================================
    print(color_list)
    color_datas = {}
    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]
        print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")
        color_datas[color_pa]='main color = '+str(color_name) + " "+ str(color_id) + " " + str(color_pa * 100)[:5] + "%"
    print(color_datas)
    print(color_datas.keys())
    max_data = max(color_datas.keys())
    color_data = color_datas[max_data]
    print(color_data)

    #同時存在一個，上限五則訊息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id+"\n"+color_data))
    #擷取tag訊息
    dn = "ID_message/user_event.txt"
    user_event = {}
    with open(dn, "a") as myfile:
        user_event["user_id"] = vars(user_profile)["user_id"]
        user_event["tag"] = "Image has Upload"+ " " + event.message.id+"\n"+color_data
        user_event["time"] = event.timestamp
        user_event["product_ID"] = "now to choose"
        myfile.write(json.dumps(user_event,sort_keys=True))
        myfile.write('\r\n')

In [8]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.18.0.2 - - [20/Oct/2019 09:06:56] "POST / HTTP/1.1" 200 -
172.18.0.2 - - [20/Oct/2019 09:06:58] "POST / HTTP/1.1" 200 -


{'folder': ['問題二'], 'tag': ['man']}


172.18.0.2 - - [20/Oct/2019 09:07:02] "POST / HTTP/1.1" 200 -
172.18.0.2 - - [20/Oct/2019 09:07:03] "POST / HTTP/1.1" 200 -


{'folder': ['問題三'], 'tag': ['3000']}


172.18.0.2 - - [20/Oct/2019 09:07:05] "POST / HTTP/1.1" 200 -
172.18.0.2 - - [20/Oct/2019 09:07:06] "POST / HTTP/1.1" 200 -


...

172.18.0.2 - - [20/Oct/2019 09:13:51] "POST / HTTP/1.1" 200 -
172.18.0.2 - - [20/Oct/2019 09:13:52] "POST / HTTP/1.1" 200 -


{'folder': ['流行趨勢'], 'tag': ['trend']}


172.18.0.2 - - [20/Oct/2019 09:51:46] "POST / HTTP/1.1" 200 -


{'folder': ['流行趨勢'], 'tag': ['trend']}


172.18.0.2 - - [20/Oct/2019 09:51:48] "POST / HTTP/1.1" 200 -


In [9]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

'\n\nApplication 運行（heroku版）\n\n'

## TEST

### 擷取tag值

In [10]:
# h={"data": "folder=問題二&tag=woman"}
# dn = "ID_message/"+ "harry hung.json"
# with open(dn, 'r') as f:
#     print(f)
#     data = json.load(f)
#     print(data)
#     print(data["folder"])
#     print(h)
#     data["folder"].append(h["data"])
#     print(data)
#     data_1 =data
# with open(dn, 'w') as f:
#     json.dump(data_1, f)
#     print(data_1)

### 從tag取值並輸出輪播

In [11]:
#pip install random

###  資料格式為json

In [12]:
# import pandas as pd
# import random
# #準備空的list
# list_shoes = []
# #到mysql取值
# EE6461 = {
#     "商品網址":"https://www.adidas.com/us/ozweego-shoes/EE6461.html",
#     "商品照片":"https://assets.adidas.com/images/w_600,f_auto,q_auto/85c5a8551bc64690a5aaaa9300cbe16e_9366/OZW",
#     "商品標題":"adidas OZWEEGO Shoes - Green | adidas US",
#     "品牌":"adidas",
#     "分類":"Lifestyle"
# }
# G54001 = {
#     "商品網址":"https://www.adidas.com/us/ultraboost-shoes/G54001.html",
#     "商品照片":"https://assets.adidas.com/images/w_600,f_auto,q_auto/98a870193e9341b0af19aa76011ef0b8_9366/Ultra",
#     "商品標題":"adidas Ultraboost Shoes - Black | adidas US",
#     "品牌":"adidas",
#     "分類":"Running"
# }
# EF2508 = {
#     "商品網址":"https://www.adidas.com/us/americana-low-shoes/EF2508.html",
#     "商品照片":"https://assets.adidas.com/images/w_600,f_auto,q_auto/087d3c85ea0b47309305a9f201416485_9366/Ame",
#     "商品標題":"adidas Americana Low Shoes - White | adidas US",
#     "品牌":"adidas",
#     "分類":"Lifestyle"
# }

# list_shoes.append(EE6461)
# list_shoes.append(G54001)
# list_shoes.append(EF2508)
# print(list_shoes)
# random.shuffle(list_shoes)
# print(list_shoes)
# #準備空的list
# list_col_color = []
# #把值輸入col_1
# number = len(list_shoes)
# for i in range(0, number):
#     #輸入值
#     web_pic = list_shoes[i]['商品照片']
#     main_color = list_shoes[i]['品牌']
#     second_color = list_shoes[i]['分類']
#     web_shoes = list_shoes[i]['商品網址']
#     name_shoes = list_shoes[i]['商品標題']

#     #循環加到 list_col_color
#     col_1 = {
#             "thumbnailImageUrl": web_pic,
#             "title": "所顏是否?",
#             "text": main_color,
#             "actions": [
#               {
#                 "type": "postback",
#                 "label": second_color,
#                 "text": web_shoes,
#                 "data": "tag=maincolor|Secondarycolor|"+name_shoes
#               }
#             ]
#           }
#     list_col_color.append(col_1)
# print(list_col_color)

# #輸出到linebot輪播
# dc = '素材/商品推薦/reply.json'
# data =[
#     {
#     "type": "template",
#     "altText": "this is a carousel template",
#     "template":
#     {
#         "type": "carousel",
#         "actions": [],
#         "columns": list_col_color
#     }
#     }
# ]

# with open(dc, "w") as f:
#     json.dump(data, f)

###  資料格式為csv

In [13]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import pandas as pd
df = pd.read_csv('database/adidas.csv', encoding="utf-8")
list_shoes_pic = df[["商品型號","商品網址","商品照片","品牌","商品標題"]]#EE6461
list_shoes_pic.sample(6)
list_shoes = list_shoes_pic.sample(6)
shoes_data = np.array(list_shoes)#np.ndarray()
list_shoes=shoes_data.tolist()#list
print(list_shoes)
number = len(list_shoes)
print(number)

[['EE5614', 'https://www.adidas.com/us/nizza-shoes/EE5614.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/236886ebcbb54ca2b82faa78010ed975_9366/Nizza_Shoes_Purple_EE5614_01_standard.jpg', 'adidas', 'adidas Nizza Shoes - Purple | adidas US'], ['EE5910', 'https://www.adidas.com/us/nite-jogger-shoes/EE5910.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/16b833a77fa74124babcaa9300cb8b7f_9366/Nite_Jogger_Shoes_White_EE5910_01_standard.jpg', 'adidas', 'adidas Nite Jogger Shoes - White | adidas US'], ['EE5900', 'https://www.adidas.com/us/lxcon-shoes/EE5900.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/45487b3186c04d0cb882aa3f00904b2d_9366/LXCON_Shoes_Black_EE5900_01_standard.jpg', 'adidas', 'adidas LXCON Shoes - Black | adidas US'], ['BC0639', 'https://www.adidas.com/us/five-ten-freerider-pro-mountain-bike-shoes/BC0639.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/9ce0bff5214d42d586bda9af0026f90f_9366/Five_Ten_Freerider_Pro_Mountain_Bike

In [15]:
import numpy as np
import pandas as pd
df = pd.read_csv('database/adidas.csv', encoding="utf-8")
list_shoes_pic = df[["商品型號","商品網址","商品照片","品牌","商品標題"]]#EE6461
list_shoes = list_shoes_pic.sample(7)
shoes_data = np.array(list_shoes)#np.ndarray()
list_shoes=shoes_data.tolist()#list
print(list_shoes)
number = len(list_shoes)
print(number)
for i in range(0, number):
    #輸入值
#   main_color, web_shoes, web_pic, name_shoes = list_shoes[i]
    web_shoes = list_shoes[i][1]
    web_pic = list_shoes[i][2]
    main_color = list_shoes[i][0]
    second_color = list_shoes[i][3]
    name_shoes = list_shoes[i][4]

    #循環加到 list_col_color
    col_1 = {
            "thumbnailImageUrl": web_pic,
            "title": "所顏是否?",
            "text": main_color,
            "actions": [
              {
                "type": "postback",
                "label": second_color,
                "text": web_shoes,
                "data": "tag=maincolor|Secondarycolor|"+name_shoes
              }
            ]
          }
    list_col_color.append(col_1)
print(list_col_color)

#輸出到linebot輪播
dc = '素材/商品推薦/reply.json'
data =[
    {
    "type": "template",
    "altText": "this is a carousel template",
    "template":
    {
        "type": "carousel",
        "actions": [],
        "columns": list_col_color
    }
    }
]

with open(dc, "w") as f:
    json.dump(data, f)

[['F34124', 'https://www.adidas.com/us/nite-jogger-shoes/F34124.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/ec4d46c8c43544a89990a9fa0127ee88_9366/Nite_Jogger_Shoes_White_F34124_01_standard.jpg', 'adidas', 'adidas Nite Jogger Shoes - White | adidas US'], ['EE7077', 'https://www.adidas.com/us/kamanda-shoes/EE7077.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/abcb717d2196492a86c4aac000ca72e2_9366/Kamanda_Shoes_White_EE7077_01_standard.jpg', 'adidas', 'adidas Kamanda Shoes - White | adidas US'], ['F34429', 'https://www.adidas.com/us/nemeziz-19.3-turf-shoes/F34429.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/37e905dedcc4408cb323aa24009562d2_9366/Nemeziz_19.3_Turf_Shoes_Blue_F34429_01_standard.jpg', 'adidas', 'adidas Nemeziz 19.3 Turf Shoes - Blue | adidas US'], ['EG1687', 'https://www.adidas.com/us/white-mountaineering-nite-jogger-shoes/EG1687.html', 'https://assets.adidas.com/images/w_600,f_auto,q_auto/7e52c35faf09484883daaab80115db95_9366/Wh

NameError: name 'list_col_color' is not defined

In [16]:
pip install kafka

     |████████████████████████████████| 215kB 250kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
